In [ ]:
import pandas as pd
from geopy.distance import geodesic
from itertools import combinations

# Update the file path to match your local directory
file_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"

# Load the dataset (modify sheet name if necessary)
df = pd.read_excel(file_path, sheet_name="Sheet1")  # Change sheet name if needed

# Ensure required columns are present
required_columns = {'ID', 'Latitude', 'Longitude'}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Missing required columns: {required_columns - set(df.columns)}")

print("Columns found. Proceeding with distance calculations...")

# Create an edge list
edge_list = []

# Compute all-to-all distances
for (id1, lat1, lon1), (id2, lat2, lon2) in combinations(df[['ID', 'Latitude', 'Longitude']].values, 2):
    distance_km = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    edge_list.append([id1, id2, distance_km])
    edge_list.append([id2, id1, distance_km])  # Add reverse direction

# Convert to DataFrame
edge_df = pd.DataFrame(edge_list, columns=["From", "To", "Dist_Km"])

# Save to CSV
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_node_edgelist.csv"
edge_df.to_csv(output_path, index=False)

print(f"Edge list with distances saved as '{output_path}'.")


In [1]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
from itertools import product

# File paths
sink_nodes_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"
candidate_nodes_path = r"C:\Users\himu1\Desktop\project\Project1\Databases_Sreekanta\Databases_Sreekanta\random_points1000.shp"
class_ii_shut_in_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Databases_Sreekanta\LA_NETL_SONRIS_VI_Dry.shp"
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_edgelist.csv"

# Load sink nodes from Excel
df_sinks = pd.read_excel(sink_nodes_path, sheet_name="Sheet1")

# Load candidate nodes from shapefiles
gdf_candidates_1 = gpd.read_file(candidate_nodes_path)  # Random points
gdf_candidates_2 = gpd.read_file(class_ii_shut_in_path)  # Class II Shut-in Dry Hole

# Ensure required columns exist
required_columns = {'ID', 'Latitude', 'Longitude'}
if not required_columns.issubset(df_sinks.columns):
    raise ValueError(f"Missing columns in sink dataset: {required_columns - set(df_sinks.columns)}")
if 'geometry' not in gdf_candidates_1.columns or 'geometry' not in gdf_candidates_2.columns:
    raise ValueError("Missing geometry column in candidate shapefiles")

# Extract latitude and longitude from geometry
gdf_candidates_1 = gdf_candidates_1.to_crs(epsg=4326)
gdf_candidates_2 = gdf_candidates_2.to_crs(epsg=4326)

gdf_candidates_1['Latitude'] = gdf_candidates_1.geometry.y
gdf_candidates_1['Longitude'] = gdf_candidates_1.geometry.x
gdf_candidates_2['Latitude'] = gdf_candidates_2.geometry.y
gdf_candidates_2['Longitude'] = gdf_candidates_2.geometry.x

# Use latitude from sink_nodes_path for consistency
df_candidates = pd.concat([
    gdf_candidates_1[['ID', 'Latitude', 'Longitude']],
    gdf_candidates_2[['ID', 'Latitude', 'Longitude']]
], ignore_index=True)

# Ensure latitude values are within valid range
df_candidates = df_candidates[(df_candidates['Latitude'] >= -90) & (df_candidates['Latitude'] <= 90)]

df_sinks = df_sinks[(df_sinks['Latitude'] >= -90) & (df_sinks['Latitude'] <= 90)]

# Create an edge list
edge_list = []

# Compute distances from each sink to each candidate
for (sink_id, sink_lat, sink_lon), (cand_id, cand_lat, cand_lon) in product(
    df_sinks[['ID', 'Latitude', 'Longitude']].values,
    df_candidates[['ID', 'Latitude', 'Longitude']].values
):
    distance_km = geodesic((sink_lat, sink_lon), (cand_lat, cand_lon)).kilometers
    edge_list.append([sink_id, cand_id, distance_km, "Candidate"])

# Convert to DataFrame
edge_df = pd.DataFrame(edge_list, columns=["From", "To", "Dist_Km", "Type"])

# Save to CSV
edge_df.to_csv(output_path, index=False)

print(f"Edge list with distances saved as '{output_path}'.")


Edge list with distances saved as 'C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_edgelist.csv'.


In [ ]:
import pandas as pd
from geopy.distance import geodesic
from itertools import product

# File paths
sink_candidate_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_edgelist.csv"
ejs_data_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"
sink_nodes_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"
candidate_nodes_path = r"C:\Users\himu1\Desktop\project\Project1\Databases_Sreekanta\Databases_Sreekanta\random_points1000.shp"
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_population_edgelist.csv"

# Load sink + candidate node distances
df_sinks_candidates = pd.read_csv(sink_candidate_path)

# Load sink nodes
df_sinks = pd.read_excel(sink_nodes_path, sheet_name="Sheet1")

# Load EJS data from Excel
df_ejs = pd.read_excel(ejs_data_path, sheet_name="Sheet1")

# Print available columns to debug column names
print("Available columns in EJS dataset:", df_ejs.columns)

# Ensure required columns exist
required_columns_xlsx = {'TractID', 'Latitude', 'Longitude', 'TractLandm', 'TotPop'}
if not required_columns_xlsx.issubset(df_ejs.columns):
    raise ValueError(f"Missing columns in EJS dataset: {required_columns_xlsx - set(df_ejs.columns)}")

# Merge latitude and longitude into sink-candidate dataset
df_sinks_candidates = df_sinks_candidates.merge(df_sinks[['ID', 'Latitude', 'Longitude']], left_on='From', right_on='ID', how='left')

# Filter population nodes within 30 miles
def within_30_miles(lat1, lon1, lat2, lon2):
    return geodesic((lat1, lon1), (lat2, lon2)).miles <= 30

# Create an edge list
edge_list = []

# Compute distances from each sink/candidate to population node
for (node_id, node_lat, node_lon), (pop_id, pop_lat, pop_lon, tract_land, tot_pop) in product(
    df_sinks_candidates[['From', 'Latitude', 'Longitude']].values,
    df_ejs[['TractID', 'Latitude', 'Longitude', 'TractLandm', 'TotPop']].values
):
    if within_30_miles(node_lat, node_lon, pop_lat, pop_lon):
        distance_km = geodesic((node_lat, node_lon), (pop_lat, pop_lon)).kilometers
        edge_list.append([node_id, pop_id, distance_km, tract_land, tot_pop])

# Convert to DataFrame
edge_df = pd.DataFrame(edge_list, columns=["From", "To", "Dist_Km", "TractLandm", "TotPop"])

# Save to CSV
edge_df.to_csv(output_path, index=False)

print(f"Edge list with population distances saved as '{output_path}'.")


In [ ]:
import pandas as pd
from geopy.distance import geodesic
from itertools import product

# File paths
ejs_data_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_population_edgelist.csv"

# Load EJS data from Excel
df_ejs = pd.read_excel(ejs_data_path, sheet_name="Sheet1")

# Print available columns to debug column names
print("Available columns in EJS dataset:", df_ejs.columns)

# Rename columns to match expected names
df_ejs.rename(columns={
    'EJS_TractID': 'TractID',
    'EJS_TractLandm': 'TractLandm',
    'EJS_TotPop': 'TotPop'
}, inplace=True)

# Ensure required columns exist
required_columns_xlsx = {'ID', 'Latitude', 'Longitude', 'TractID', 'TractLandm', 'TotPop'}
if not required_columns_xlsx.issubset(df_ejs.columns):
    raise ValueError(f"Missing columns in EJS dataset: {required_columns_xlsx - set(df_ejs.columns)}")

# Create an edge list
edge_list = []

# Compute distances from each sink/candidate node (ID) to each census tract (TractID) within 30 miles
for (id1, lat1, lon1), (tract_id, lat2, lon2, tract_land, tot_pop) in product(
    df_ejs[['ID', 'Latitude', 'Longitude']].values,
    df_ejs[['TractID', 'Latitude', 'Longitude', 'TractLandm', 'TotPop']].values
):
    distance_km = geodesic((lat1, lon1), (lat2, lon2)).kilometers
    if distance_km <= 48.28:  # 30 miles in kilometers
        edge_list.append([id1, tract_id, distance_km, tract_land, tot_pop])

# Convert to DataFrame
edge_df = pd.DataFrame(edge_list, columns=["From", "To", "Dist_Km", "TractLandm", "TotPop"])

# Save to CSV
edge_df.to_csv(output_path, index=False)

print(f"Edge list with computed distances saved as '{output_path}'")

In [ ]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
import os
from shapely.geometry import Point

# File paths
ejs_data_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"
source_data_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Databases_Sreekanta\LA_CO2_Source.shp"
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_source_edgelist.csv"

try:
    # Load EJS data (From locations)
    print("Loading From location data...")
    df_ejs = pd.read_excel(ejs_data_path, sheet_name="Sheet1", engine="openpyxl")
    print(f"From location data loaded: {len(df_ejs)} rows")
    
    # Load Source Nodes data (To locations)
    print("Loading To location data...")
    gdf_source = gpd.read_file(source_data_path)
    print(f"To location data loaded: {len(gdf_source)} rows")
    
    # Print column names to identify the correct ones
    print("From location columns:", df_ejs.columns.tolist())
    print("To location columns:", gdf_source.columns.tolist())
    
    # First determine if required columns exist - either directly or needing renaming
    # For From locations
    id_column = 'ID' if 'ID' in df_ejs.columns else None
    if not id_column and 'From' in df_ejs.columns:
        id_column = 'From'
    
    lat_column_ejs = 'Latitude' if 'Latitude' in df_ejs.columns else None
    lon_column_ejs = 'Longitude' if 'Longitude' in df_ejs.columns else None
    
    # For To locations
    source_id_column = None
    for possible_name in ['GHGRP_ID', 'To', 'ID', 'FID']:
        if possible_name in gdf_source.columns:
            source_id_column = possible_name
            break
    
    co2_column = None
    for possible_name in ['GHG_QUANTI', 'TonsCO2e', 'CO2', 'Emissions']:
        if possible_name in gdf_source.columns:
            co2_column = possible_name
            break
    
    # Validate we found all required columns
    missing_columns = []
    if not id_column:
        missing_columns.append("ID/From column in From location data")
    if not lat_column_ejs:
        missing_columns.append("Latitude column in From location data")
    if not lon_column_ejs:
        missing_columns.append("Longitude column in From location data")
    if not source_id_column:
        missing_columns.append("ID/To column in To location data")
    if not co2_column:
        missing_columns.append("CO2 emissions column in To location data")
    
    if missing_columns:
        raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
    
    # Create standardized column names
    df_ejs = df_ejs.rename(columns={id_column: 'From'})
    gdf_source = gdf_source.rename(columns={source_id_column: 'To', co2_column: 'TonsCO2e'})
    
    # Make sure coordinates are numeric
    df_ejs[lat_column_ejs] = pd.to_numeric(df_ejs[lat_column_ejs], errors='coerce')
    df_ejs[lon_column_ejs] = pd.to_numeric(df_ejs[lon_column_ejs], errors='coerce')
    
    # Drop rows with missing coordinates
    before_drop = len(df_ejs)
    df_ejs.dropna(subset=[lat_column_ejs, lon_column_ejs], inplace=True)
    after_drop = len(df_ejs)
    print(f"Dropped {before_drop - after_drop} rows with missing coordinates in From location data")
    
    # Calculate centroids for To locations
    print("Calculating centroids for To locations...")
    # If geometry is not already Point type, calculate centroids
    if not all(isinstance(geom, Point) for geom in gdf_source.geometry):
        gdf_source['centroid'] = gdf_source.geometry.centroid
        gdf_source['centroid_lat'] = gdf_source.centroid.y
        gdf_source['centroid_lon'] = gdf_source.centroid.x
    else:
        # If already Point type, just extract coordinates
        gdf_source['centroid_lat'] = gdf_source.geometry.y
        gdf_source['centroid_lon'] = gdf_source.geometry.x
    
    # Create the edge list with distances
    print("Creating edge list with distances...")
    edge_list = []
    
    # Total combinations to process
    total_pairs = len(df_ejs) * len(gdf_source)
    print(f"Processing {total_pairs} connections...")
    
    # Calculate distances and build edge list
    pairs_processed = 0
    
    # Process in chunks to show progress
    chunk_size = max(1, min(10000, total_pairs // 10))
    
    for i, sink_row in df_ejs.iterrows():
        sink_id = sink_row['From']
        sink_lat = sink_row[lat_column_ejs]
        sink_lon = sink_row[lon_column_ejs]
        
        # Skip if missing coordinate data
        if pd.isna(sink_lat) or pd.isna(sink_lon):
            continue
        
        for j, source_row in gdf_source.iterrows():
            pairs_processed += 1
            
            # Show progress
            if pairs_processed % chunk_size == 0:
                progress_pct = (pairs_processed / total_pairs) * 100
                print(f"Progress: {progress_pct:.1f}% ({pairs_processed}/{total_pairs})")
            
            source_id = source_row['To']
            source_lat = source_row['centroid_lat']
            source_lon = source_row['centroid_lon']
            tons_co2e = source_row['TonsCO2e']
            
            # Skip if missing coordinate data
            if pd.isna(source_lat) or pd.isna(source_lon) or pd.isna(tons_co2e):
                continue
                
            # Validate coordinates are in proper range
            if (-90 <= sink_lat <= 90 and -180 <= sink_lon <= 180 and 
                -90 <= source_lat <= 90 and -180 <= source_lon <= 180):
                
                try:
                    # Calculate distance
                    distance_km = geodesic(
                        (sink_lat, sink_lon), 
                        (source_lat, source_lon)
                    ).kilometers
                    
                    # Add all connections regardless of distance
                    edge_list.append({
                        'From': sink_id,
                        'To': source_id,
                        'Dist_Km': distance_km,
                        'TonsCO2e': tons_co2e,
                        'Type': "Source"
                    })
                except Exception as e:
                    print(f"Error calculating distance: {e} for points ({sink_lat},{sink_lon}) and ({source_lat},{source_lon})")
    
    # Convert to DataFrame
    edge_df = pd.DataFrame(edge_list)
    
    # Check if we found any connections
    if edge_df.empty:
        print("WARNING: No connections created!")
        print("This could indicate a problem with the input data.")
        
        # Create an empty CSV with the correct columns
        empty_df = pd.DataFrame(columns=['From', 'To', 'Dist_Km', 'TonsCO2e', 'Type'])
        empty_df.to_csv(output_path, index=False)
        print(f"Empty CSV with required columns created at: {output_path}")
    else:
        print(f"Created {len(edge_df)} connections")
        
        # Make sure all required columns are present and in the right order
        result_columns = ['From', 'To', 'Dist_Km', 'TonsCO2e', 'Type']
        for col in result_columns:
            if col not in edge_df.columns:
                if col == 'Type':
                    # Add Type column if missing
                    edge_df['Type'] = 'Source'
        
        # Reorder columns to ensure they're in the expected order
        edge_df = edge_df[result_columns]
        
        # Display information about the output
        print("\nColumns in the final CSV:")
        for col in edge_df.columns:
            print(f"- {col}: {edge_df[col].dtype}")
        
        print("\nSample of first 5 rows:")
        print(edge_df.head())
        
        # Distance statistics
        print("\nDistance statistics (km):")
        print(f"Min: {edge_df['Dist_Km'].min():.2f}")
        print(f"Max: {edge_df['Dist_Km'].max():.2f}")
        print(f"Mean: {edge_df['Dist_Km'].mean():.2f}")
        
        # Save to CSV - ensure all columns are exported
        try:
            edge_df.to_csv(output_path, index=False)
            print(f"\nCSV file successfully created at: {output_path}")
            print(f"File contains {len(edge_df)} rows and {len(edge_df.columns)} columns")
            
            # Verify file was created
            if os.path.exists(output_path):
                file_size = os.path.getsize(output_path) / 1024  # Size in KB
                print(f"File size: {file_size:.2f} KB")
            else:
                print("Warning: File does not appear to exist after saving!")
        except Exception as e:
            print(f"Error saving to CSV: {e}")
            
            # Try an alternative location
            alt_path = os.path.join(os.path.expanduser("~"), "Desktop", "sink_source_edgelist_backup.csv")
            edge_df.to_csv(alt_path, index=False)
            print(f"Saved to alternative location: {alt_path}")

except Exception as e:
    import traceback
    print(f"ERROR: {e}")
    print(traceback.format_exc())
    
    # Try to create a minimal CSV with the required columns even if processing failed
    try:
        minimal_df = pd.DataFrame(columns=['From', 'To', 'Dist_Km', 'TonsCO2e', 'Type'])
        backup_path = os.path.join(os.path.dirname(output_path), "empty_edge_list.csv")
        minimal_df.to_csv(backup_path, index=False)
        print(f"Created empty CSV template at: {backup_path}")
    except:
        print("Could not create empty CSV template")

In [ ]:
import pandas as pd
import geopandas as gpd
from geopy.distance import geodesic
from itertools import product

# File paths
ejs_data_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"
pipeline_data_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Databases_Sreekanta\LA_CO2Pipe.shp"
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_pipeline_edgelist.csv"

# Load EJS data from Excel using openpyxl
df_ejs = pd.read_excel(ejs_data_path, sheet_name="Sheet1", engine="openpyxl")

# Load Pipeline Nodes data using GeoPandas
gdf_pipeline = gpd.read_file(pipeline_data_path)

# Print available columns to debug column names
print("Available columns in EJS dataset:", df_ejs.columns)
print("Available columns in Pipeline dataset:", gdf_pipeline.columns)

# Rename columns to match expected names
df_ejs.rename(columns={'ID': 'From'}, inplace=True)
if 'PLINE_ID' in gdf_pipeline.columns:
    gdf_pipeline.rename(columns={'PLINE_ID': 'To'}, inplace=True)
    gdf_pipeline['To'] = gdf_pipeline['To'].astype(str)  # Ensure To column is string
else:
    raise ValueError("PLINE_ID column not found in Pipeline dataset. Check correct column name.")

# Convert Latitude/Longitude to numeric
df_ejs['Latitude'] = pd.to_numeric(df_ejs['Latitude'], errors='coerce')
df_ejs['Longitude'] = pd.to_numeric(df_ejs['Longitude'], errors='coerce')

# Drop rows with missing Latitude/Longitude
df_ejs.dropna(subset=['Latitude', 'Longitude'], inplace=True)

# Extract Latitude and Longitude from geometry in pipeline dataset
gdf_pipeline['Latitude'] = gdf_pipeline.geometry.centroid.y
gdf_pipeline['Longitude'] = gdf_pipeline.geometry.centroid.x

# Debugging Step: Print pipeline data to ensure values exist
print("Sample Pipeline Data:")
print(gdf_pipeline[['To', 'Latitude', 'Longitude']].head())

# Ensure required columns exist
required_columns_ejs = {'From', 'Latitude', 'Longitude'}
required_columns_pipeline = {'To', 'Latitude', 'Longitude'}
if not required_columns_ejs.issubset(df_ejs.columns):
    raise ValueError(f"Missing columns in EJS dataset: {required_columns_ejs - set(df_ejs.columns)}")
if not required_columns_pipeline.issubset(gdf_pipeline.columns):
    raise ValueError(f"Missing columns in Pipeline dataset: {required_columns_pipeline - set(gdf_pipeline.columns)}")

# Create an edge list
edge_list = []

# Compute distances from each sink/candidate node (From) to each pipeline node (To)
for (sink_id, lat1, lon1), (pipeline_id, lat2, lon2) in product(
    df_ejs[['From', 'Latitude', 'Longitude']].values,
    gdf_pipeline[['To', 'Latitude', 'Longitude']].values
):
    if -90 <= lat1 <= 90 and -90 <= lat2 <= 90:
        distance_km = geodesic((lat1, lon1), (lat2, lon2)).kilometers
        edge_list.append([sink_id, pipeline_id, distance_km, "Pipeline"])

# Convert to DataFrame
edge_df = pd.DataFrame(edge_list, columns=["From", "To", "Dist_Km", "Type"])

# Save to CSV
edge_df.to_csv(output_path, index=False)

print(f"Edge list with computed distances saved as '{output_path}'")


In [ ]:
# Import necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import matplotlib.pyplot as plt
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # suppress SettingWithCopyWarning

# Define file paths (update these with your correct paths)
sink_candidate_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"
source_shapefile_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Databases_Sreekanta\LA_CO2_Source.shp"

# Load the sink/candidate nodes data - corrected to use read_excel instead of read_csv
try:
    sink_candidate_df = pd.read_excel(sink_candidate_path)
except Exception as e:
    print(f"Error loading excel file: {e}")
    # Fallback to CSV in case the file was saved as CSV despite the .xlsx extension
    sink_candidate_df = pd.read_csv(sink_candidate_path)

# Check and use LATITUDE and LONGITUDE columns in source_gdf if geometry is not already present
if 'geometry' not in source_gdf.columns and 'LATITUDE' in source_gdf.columns and 'LONGITUDE' in source_gdf.columns:
    print("Creating geometry for source data using LATITUDE and LONGITUDE columns")
    source_gdf = source_gdf.dropna(subset=['LATITUDE', 'LONGITUDE'])
    geometry = [Point(xy) for xy in zip(source_gdf['LONGITUDE'], source_gdf['LATITUDE'])]
    source_gdf = gpd.GeoDataFrame(source_gdf, geometry=geometry, crs="EPSG:4326")

# Check columns to ensure they exist
print("Sink candidate columns:", sink_candidate_df.columns.tolist())
print("Source columns:", source_gdf.columns.tolist())

# Make sure both dataframes have geometry columns
# If sink_candidate_df doesn't have geometry, create it from lat/lon columns
if 'geometry' not in sink_candidate_df.columns:
    # Use the exact column names specified for sink_candidate_df
    if 'Latitude' in sink_candidate_df.columns and 'Longitude' in sink_candidate_df.columns:
        # Filter out any rows with missing coordinates
        sink_candidate_df = sink_candidate_df.dropna(subset=['Latitude', 'Longitude'])
        geometry = [Point(xy) for xy in zip(sink_candidate_df['Longitude'], sink_candidate_df['Latitude'])]
        sink_candidate_gdf = gpd.GeoDataFrame(sink_candidate_df, geometry=geometry, crs="EPSG:4326")
    else:
        raise ValueError(f"Could not find 'Latitude' and 'Longitude' columns in sink candidate data. Available columns: {sink_candidate_df.columns.tolist()}")
else:
    sink_candidate_gdf = gpd.GeoDataFrame(sink_candidate_df, crs="EPSG:4326")

# Ensure both GeoDataFrames use the same coordinate reference system
print(f"Sink CRS: {sink_candidate_gdf.crs}, Source CRS: {source_gdf.crs}")
if sink_candidate_gdf.crs != source_gdf.crs:
    sink_candidate_gdf = sink_candidate_gdf.to_crs(source_gdf.crs)

# Function to calculate distance in kilometers
def calculate_distance_km(point1, point2):
    # Make sure we're working with GeoSeries objects
    if not isinstance(point1, gpd.GeoSeries):
        point1 = gpd.GeoSeries([point1], crs=sink_candidate_gdf.crs)
    if not isinstance(point2, gpd.GeoSeries):
        point2 = gpd.GeoSeries([point2], crs=source_gdf.crs)
    
    # Convert to projected CRS for accurate distance calculation
    if point1.crs.is_geographic:
        point1 = point1.to_crs(epsg=3857)  # Web Mercator projection
        point2 = point2.to_crs(epsg=3857)
    
    # Calculate distance in meters and convert to kilometers
    distance_m = point1.distance(point2).values[0]  # Extract the value from the series
    return distance_m / 1000

# Use 'ID' column as the identifier for sink/candidate nodes
if 'ID' in sink_candidate_gdf.columns:
    print("Using 'ID' column as sink identifier")
    sink_candidate_gdf['From'] = sink_candidate_gdf['ID']
else:
    # Fallback if 'ID' column doesn't exist
    id_col = next((col for col in sink_candidate_gdf.columns if 'id' in col.lower()), None)
    if id_col:
        print(f"Using '{id_col}' as sink ID column")
        sink_candidate_gdf['From'] = sink_candidate_gdf[id_col]
    else:
        print("Creating sequential IDs for sinks")
        sink_candidate_gdf['From'] = [f"Sink_{i}" for i in range(len(sink_candidate_gdf))]

# Check for GHGRP_ID in source data
source_id_col = 'GHGRP_ID'
if source_id_col not in source_gdf.columns:
    source_id_col = next((col for col in source_gdf.columns if 'id' in col.lower()), None)
    if not source_id_col:
        print("Creating sequential IDs for sources")
        source_gdf['GHGRP_ID'] = [f"Source_{i}" for i in range(len(source_gdf))]
        source_id_col = 'GHGRP_ID'
    else:
        print(f"Using '{source_id_col}' as source ID column")

# Check for emissions column
emission_col = 'GHG_QUANTI'
if emission_col not in source_gdf.columns:
    emission_col = next((col for col in source_gdf.columns if 'co2' in col.lower() or 'ghg' in col.lower() or 'emission' in col.lower() or 'quanti' in col.lower()), None)
    if not emission_col:
        print("No emissions data found, using placeholder values")
        source_gdf[emission_col] = 1000  # Default placeholder value
    else:
        print(f"Using '{emission_col}' as emissions column")
else:
    print(f"Using '{emission_col}' as emissions column")

# Create empty lists to store edge data
from_nodes = []
to_nodes = []
distances_km = []
co2_tons = []

# Define radius in kilometers (100 miles ≈ 160.9344 kilometers)
radius_km = 160.9344

# For each sink/candidate node, find sources within radius
for idx, sink in sink_candidate_gdf.iterrows():
    sink_id = sink['From']
    sink_point = sink.geometry
    
    # Find sources within radius
    for src_idx, source in source_gdf.iterrows():
        source_id = source[source_id_col]
        source_point = source.geometry
        
        try:
            # Calculate distance
            dist_km = calculate_distance_km(sink_point, source_point)
            
            # If within radius, add to edge list
            if dist_km <= radius_km:
                from_nodes.append(sink_id)
                to_nodes.append(source_id)
                distances_km.append(dist_km)
                co2_tons.append(source[emission_col])
        except Exception as e:
            print(f"Error calculating distance between {sink_id} and {source_id}: {e}")
            continue

# Create edge list dataframe
edge_list_df = pd.DataFrame({
    'From': from_nodes,
    'To': to_nodes,
    'Relational process description': 'Sink/candidate node to source node within 100 miles',
    'Edge Attribute': 'Distance and CO2 emissions',
    'Column Names': 'Dist_Km, TonsCO2e',
    'Type': 'Source',
    'Dist_Km': distances_km,
    'TonsCO2e': co2_tons
})

# Save the edge list to a CSV file
output_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_source_edge_list.csv"
edge_list_df.to_csv(output_path, index=False)

# Display the first few rows of the edge list
print(edge_list_df.head())
print(f"Total connections found: {len(edge_list_df)}")

# Visualization section removed as requested
print("Skipping visualization portion")
print(f"Analysis complete. {len(edge_list_df)} connections found between sources and sinks.")
print(f"Edge list saved to: {output_path}")

In [ ]:
# Import necessary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # suppress SettingWithCopyWarning

# Define file paths (update these with your correct paths)
sink_candidate_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Database_with_Spatial_Analysis_final.xlsx"
pipeline_shapefile_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\Databases_Sreekanta\LA_CO2Pipe.shp"

# Load the sink/candidate nodes data
try:
    print("Loading sink/candidate data...")
    sink_candidate_df = pd.read_excel(sink_candidate_path)
except Exception as e:
    print(f"Error loading excel file: {e}")
    # Fallback to CSV in case the file was saved as CSV despite the .xlsx extension
    sink_candidate_df = pd.read_csv(sink_candidate_path)

# Load the pipeline data as GeoDataFrame
print("Loading pipeline shapefile...")
pipeline_gdf = gpd.read_file(pipeline_shapefile_path)

# Print column information for debugging
print("Sink candidate columns:", sink_candidate_df.columns.tolist())
print("Pipeline columns:", pipeline_gdf.columns.tolist())

# Check for FID column in pipeline data
pipeline_id_col = 'FID'
if pipeline_id_col not in pipeline_gdf.columns:
    # Try to find appropriate ID column
    pipeline_id_col = next((col for col in pipeline_gdf.columns 
                          if col.lower() in ['fid', 'id', 'pipeline_id', 'pipe_id']), None)
    if not pipeline_id_col:
        print("Creating sequential IDs for pipeline nodes")
        pipeline_gdf['FID'] = [f"Pipeline_{i}" for i in range(len(pipeline_gdf))]
        pipeline_id_col = 'FID'
    else:
        print(f"Using '{pipeline_id_col}' as pipeline ID column")

# Make sure both dataframes have geometry columns
# Process sink/candidate data
if 'geometry' not in sink_candidate_df.columns:
    # Use the exact column names for sink_candidate_df
    if 'Latitude' in sink_candidate_df.columns and 'Longitude' in sink_candidate_df.columns:
        # Filter out any rows with missing coordinates
        sink_candidate_df = sink_candidate_df.dropna(subset=['Latitude', 'Longitude'])
        geometry = [Point(xy) for xy in zip(sink_candidate_df['Longitude'], sink_candidate_df['Latitude'])]
        sink_candidate_gdf = gpd.GeoDataFrame(sink_candidate_df, geometry=geometry, crs="EPSG:4326")
    else:
        raise ValueError(f"Could not find 'Latitude' and 'Longitude' columns in sink candidate data. Available columns: {sink_candidate_df.columns.tolist()}")
else:
    sink_candidate_gdf = gpd.GeoDataFrame(sink_candidate_df, crs="EPSG:4326")

# Use 'ID' column as the identifier for sink/candidate nodes
if 'ID' in sink_candidate_gdf.columns:
    print("Using 'ID' column as sink identifier")
    sink_candidate_gdf['From'] = sink_candidate_gdf['ID']
else:
    # Fallback if 'ID' column doesn't exist
    id_col = next((col for col in sink_candidate_gdf.columns if 'id' in col.lower()), None)
    if id_col:
        print(f"Using '{id_col}' as sink ID column")
        sink_candidate_gdf['From'] = sink_candidate_gdf[id_col]
    else:
        print("Creating sequential IDs for sinks")
        sink_candidate_gdf['From'] = [f"Sink_{i}" for i in range(len(sink_candidate_gdf))]

# Ensure both GeoDataFrames use the same coordinate reference system
print(f"Sink CRS: {sink_candidate_gdf.crs}, Pipeline CRS: {pipeline_gdf.crs}")
if sink_candidate_gdf.crs != pipeline_gdf.crs:
    sink_candidate_gdf = sink_candidate_gdf.to_crs(pipeline_gdf.crs)

# Function to calculate distance in kilometers to a linestring (pipeline)
def calculate_distance_to_line_km(point, line):
    # Make sure we're working with GeoSeries objects
    if not isinstance(point, gpd.GeoSeries):
        point = gpd.GeoSeries([point], crs=sink_candidate_gdf.crs)
    if not isinstance(line, gpd.GeoSeries):
        line = gpd.GeoSeries([line], crs=pipeline_gdf.crs)
    
    # Convert to projected CRS for accurate distance calculation
    if point.crs.is_geographic:
        point = point.to_crs(epsg=3857)  # Web Mercator projection
        line = line.to_crs(epsg=3857)
    
    # Calculate distance in meters and convert to kilometers
    distance_m = point.distance(line).values[0]  # Extract the value from the series
    return distance_m / 1000

# Create empty lists to store edge data
from_nodes = []
to_nodes = []
distances_km = []

# For each sink/candidate node, find distance to all pipeline nodes
print("Calculating distances between sink nodes and pipeline nodes...")
total_count = len(sink_candidate_gdf) * len(pipeline_gdf)
count = 0
progress_interval = max(1, total_count // 20)  # Show progress at 5% intervals

for idx, sink in sink_candidate_gdf.iterrows():
    sink_id = sink['From']
    sink_point = sink.geometry
    
    for pipe_idx, pipeline in pipeline_gdf.iterrows():
        count += 1
        if count % progress_interval == 0:
            print(f"Progress: {count/total_count*100:.1f}% ({count}/{total_count})")
            
        pipe_id = pipeline[pipeline_id_col]
        pipe_geometry = pipeline.geometry
        
        try:
            # Calculate distance to pipeline
            dist_km = calculate_distance_to_line_km(sink_point, pipe_geometry)
            
            # Store the data
            from_nodes.append(sink_id)
            to_nodes.append(pipe_id)
            distances_km.append(dist_km)
            
            # We don't need to collect Pipeline IDs as requested
                
        except Exception as e:
            print(f"Error calculating distance between {sink_id} and {pipe_id}: {e}")
            continue

# Create edge list dataframe
print("Creating edge list...")
edge_data = {
    'From': from_nodes,
    'To': to_nodes,
    'Relational process description': 'Sink/candidate node to pipeline node',
    'Edge Attribute': 'Distance',
    'Column Names': 'Dist_Km',
    'Type': 'Pipeline',
    'Dist_Km': distances_km
}

edge_list_df = pd.DataFrame(edge_data)

# Save the edge list to a CSV file
output_path = r"C:\Users\himu1\Desktop\project\Project2\sink_pipeline_edge_list.csv"
edge_list_df.to_csv(output_path, index=False)

# Display the first few rows of the edge list
print(edge_list_df.head())
print(f"Analysis complete. {len(edge_list_df)} connections found between sink nodes and pipeline nodes.")
print(f"Edge list saved to: {output_path}")

In [ ]:
# Import necessary libraries
import pandas as pd
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # suppress SettingWithCopyWarning

# Define file paths
input_file_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_population_edgelist.csv"
output_file_path = r"C:\Users\himu1\Desktop\project\Project2\Databases_Sreekanta\sink_candidate_population_edgelist_updated.csv"

# Read the CSV file
try:
    print(f"Reading file from: {input_file_path}")
    edge_list_df = pd.read_csv(input_file_path)
    
    # Display information about the file
    print(f"File loaded successfully. Shape: {edge_list_df.shape}")
    print(f"Columns: {edge_list_df.columns.tolist()}")
    print(f"First few rows:\n{edge_list_df.head()}")
    
    # Add the Type column with "PopulationCluster" value
    edge_list_df['Type'] = 'PopulationCluster'
    
    # Save the updated CSV file
    edge_list_df.to_csv(output_file_path, index=False)
    print(f"Updated file saved to: {output_file_path}")
    
    # Show the updated data
    print(f"Updated first few rows:\n{edge_list_df.head()}")
    print(f"Total rows processed: {len(edge_list_df)}")
    
except Exception as e:
    print(f"Error processing file: {e}")

In [ ]:
# Import necessary libraries
import pandas as pd
import os
import glob
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None  # suppress SettingWithCopyWarning

# Define file paths
base_path = r"C:\Users\himu1\Desktop\project\Project2\Edge list"
output_file_path = os.path.join(base_path, "merged_edge_list.csv")

# Define the order of files to process
file_types = ["Sink", "Candidate", "Population", "Source", "Pipeline"]

# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Process each file type in the specified order
for file_type in file_types:
    # Find files that match the current type
    pattern = os.path.join(base_path, f"*{file_type}*.csv")
    matching_files = glob.glob(pattern)
    
    if not matching_files:
        print(f"Warning: No files found matching pattern '{pattern}'")
        continue
    
    # Use the first matching file
    file_path = matching_files[0]
    print(f"Processing file: {os.path.basename(file_path)}")
    
    try:
        # Read the current file
        current_df = pd.read_csv(file_path)
        
        # Display basic information about the file
        print(f"  Columns: {current_df.columns.tolist()}")
        print(f"  Rows: {len(current_df)}")
        
        # Keep track of any additional columns in this file
        additional_columns = [col for col in current_df.columns if col not in ["From", "To", "Dist_Km", "Type"]]
        
        if additional_columns:
            print(f"  Additional columns found: {additional_columns}")
        
        # If this is the first file, use it to initialize the merged DataFrame
        if merged_df.empty:
            merged_df = current_df.copy()
        else:
            # Get all columns from both DataFrames
            all_columns = set(merged_df.columns) | set(current_df.columns)
            
            # For each DataFrame, add missing columns with NA values
            for col in all_columns:
                if col not in merged_df.columns:
                    merged_df[col] = pd.NA
                if col not in current_df.columns:
                    current_df[col] = pd.NA
            
            # Append the current DataFrame to the merged one
            merged_df = pd.concat([merged_df, current_df], ignore_index=True)
        
    except Exception as e:
        print(f"  Error processing file {file_path}: {e}")

# Display information about the merged DataFrame
print("\nMerged DataFrame:")
print(f"  Total rows: {len(merged_df)}")
print(f"  All columns: {merged_df.columns.tolist()}")

# Save the merged DataFrame to a CSV file
merged_df.to_csv(output_file_path, index=False)
print(f"\nMerged edge list saved to: {output_file_path}")

# Display a sample of the merged data
print("\nSample of merged data:")
print(merged_df.head())

# Optional: Display counts by Type for verification
type_counts = merged_df['Type'].value_counts()
print("\nRows by Type:")
for type_name, count in type_counts.items():
    print(f"  {type_name}: {count}")